# Week 3: Decision Tree Mining

### What's on this week
1. [Resuming from week 2](#resume)
2. [Building your first decision tree model](#build)
3. [Understanding and visualizing your decision tree](#viz)
4. [Finding optimal hyperparameters with GridSearchCV](#gridsearch)

---

The practical note for this week introduces you to decision tree mining in Python. Decision trees are relatively simple, yet it can be powerful if build and utilised properly.

**This tutorial notes is in experimental version. Please give us feedbacks and suggestions on how to make it better. Ask your tutor for any question and clarification.**

## 1. Resuming from week 2 <a name="resume"></a>
Last week, we learned how to perform data preparation on our dataset. We build plots of data distribution, dealt with noisy values, imputed missing values and drop columns not required in our analysis. After week 2, your code should look like this:

In [1]:
# inside dm_tools.py
import numpy as np
import pandas as pd

def data_prep():
    # read the pva97nk dataset
    df = pd.read_csv('pva97nk.csv')
    
    # change DemCluster from interval/integer to nominal/str
    df['DemCluster'] = df['DemCluster'].astype(str)
    
    # change DemHomeOwner into binary 0/1 variable
    dem_home_owner_map = {'U':0, 'H': 1}
    df['DemHomeOwner'] = df['DemHomeOwner'].map(dem_home_owner_map)
    
    # denote errorneous values in DemMidIncome
    mask = df['DemMedIncome'] < 1
    df.loc[mask, 'DemMedIncome'] = np.nan
    
    # impute missing values in DemAge with its mean
    df['DemAge'].fillna(df['DemAge'].mean(), inplace=True)

    # impute med income using mean
    df['DemMedIncome'].fillna(df['DemMedIncome'].mean(), inplace=True)

    # impute gift avg card 36 using mean
    df['GiftAvgCard36'].fillna(df['GiftAvgCard36'].mean(), inplace=True)
    
    # drop ID and the unused target variable
    df.drop(['ID', 'TargetD'], axis=1, inplace=True)
    
    # one-hot encoding
    df = pd.get_dummies(df)
    
    return df

In [2]:
# in our project space
from dm_tools import data_prep

## 2. Building your first decision tree <a name="build"></a>
Before we build our decision tree, we need to set up our data partition first. Given a set of training data, you can easily build a model that is highly accurate in that dataset itself. However, a model built under this circumstance typically "overfits" the dataset, generalises poorly and has horrible accuracy in other data. To avoid this pitfall, we need to set aside some data from our whole dataset to be used as "test".

To do this, import the `train_test_split` function from `sklearn.model_selection`.

In [3]:
from sklearn.model_selection import train_test_split

After that, let's partition our data. The convention in Python is to split our data into `X` (variables used to learn and make prediction) and `y` (variable to be predicted). In our case, `y` would be `TargetB` and `X` would be the rest of the variables.

In [4]:
# preprocessing step
df = data_prep()

# train test split
y = df['TargetB']
X = df.drop(['TargetB'], axis=1)

Once you done that, convert `X` (which is still a pandas DataFrame object) into a `numpy` matrix that can be consumed by `sklearn`. Next, use the `train_test_split` function to split them into 50% training and 50% test data. We also need to stratify the split using `y` to ensure the same ratio of 0s and 1s in both train and test data set.

As `train_test_split` shuffles the dataset before splitting it, it is important to set a consistent random state (which is a seed number used to generate the shuffle). I am using 42 here, but you can use any integer number as long as it is consistent across this tutorials.

In [5]:
X_mat = X.as_matrix()
X_train, X_test, y_train, y_test = train_test_split(X_mat, y, test_size=0.5, random_state=42, stratify=y)

We are ready to build our model. Let's start by importing `DecisionTreeClassifier`. We can then train the model using `.fit` function and make predictions using `.predict`. To assess performance of the model, we could use `classification_report` and `accuracy_score` of the model's prediction on test data.

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

# simple decision tree training
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.53      0.53      0.53      2422
          1       0.53      0.52      0.52      2421

avg / total       0.53      0.53      0.53      4843

0.525294239108


## 3. Understanding and visualizing your decision tree

### 3.1. Feature importance

Let's take a deeper look on the decision tree that we just built. Firstly, we want to know what features/variables are important to the decision making process in our model. This is commonly known as "feature importance". In an `sklearn` decision tree, feature importance is stored within the model itself. Let's sort them in descending order and print them out.

In [7]:
import numpy as np

# grab feature importances from the model and feature name from the original X
importances = model.feature_importances_
feature_names = X.columns

# sort them out in descending order
indices = np.argsort(importances)
indices = np.flip(indices, axis=0)

# limit to 20 features, you can leave this out to print out everything
indices = indices[:20]

for i in indices:
    print(feature_names[i], ':', importances[i])

DemMedHomeValue : 0.0977871280501
DemMedIncome : 0.0711646902783
DemAge : 0.0618292912237
GiftAvgAll : 0.0553257239943
DemPctVeterans : 0.0547806301246
GiftAvg36 : 0.046838567038
GiftTimeFirst : 0.0458508621916
GiftTimeLast : 0.040522415804
GiftAvgLast : 0.0389083463174
PromCnt36 : 0.0381591325881
PromCntAll : 0.037392678438
GiftAvgCard36 : 0.0363760915059
PromCntCard36 : 0.0346719167253
GiftCntAll : 0.0324199698057
GiftCntCardAll : 0.0310187874853
PromCntCardAll : 0.030359431807
GiftCnt36 : 0.0291484233142
PromCnt12 : 0.0290468820979
GiftCntCard36 : 0.016404117927
DemHomeOwner : 0.00929015394395


In descending order, the top 3 important variables for this model are `DemMedHomeValue`, `DemPctVeterans` and `DemAge`. Feature importance is really important to not only understand the model, but also to learn more about the data and present conclusions to stakeholders.

### 3.2. Visualizing decision tree structure

To understand how the decision tree looks like, we could visualize it using the `export_graphviz` and `pydot`. Open the `.png` file to view the decision tree.

In [8]:
import pydot
from io import StringIO
from sklearn.tree import export_graphviz

# visualize
dotfile = StringIO()
export_graphviz(model, out_file=dotfile, feature_names=X.columns)
graph = pydot.graph_from_dot_data(dotfile.getvalue())
graph[0].write_png("week3_dt_viz.png") # saved in the following file

True

Whoops, it looks like the picture is too large and incomprehensible. This is because the model that we trained is very complex. Let's limit the complexity of the model by setting the `max_depth` that the model can go.

In [9]:
#retrain with a small max_depth limit

model = DecisionTreeClassifier(max_depth=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.55      0.72      0.62      2422
          1       0.59      0.40      0.48      2421

avg / total       0.57      0.56      0.55      4843

0.559776997729


You could see that the simpler model (small `max_depth`) actually performs much better on the test data than the complex one, showing clear example of overfitting. `max_depth` on a decision tree is what we call a hyperparameter (or just parameter) and they are responsible for the structure of a model. Different combinations of parameters will produce different models with different performance too.

Let's do a feature importance analysis and visualization on this new decision tree.

In [10]:
# grab feature importances from the model and feature name from the original X
importances = model.feature_importances_
feature_names = X.columns

# sort them out in descending order
indices = np.argsort(importances)
indices = np.flip(indices, axis=0)

# limit to 20 features, you can leave this out to print out everything
indices = indices[:20]

for i in indices:
    print(feature_names[i], ':', importances[i])

# visualize
dotfile = StringIO()
export_graphviz(model, out_file=dotfile, feature_names=X.columns)
graph = pydot.graph_from_dot_data(dotfile.getvalue())
graph[0].write_png("week3_dt_viz.png") # saved in the following file

GiftAvgLast : 0.49832551516
DemMedHomeValue : 0.179905317927
GiftAvgCard36 : 0.125956067503
GiftTimeLast : 0.103066735755
GiftCnt36 : 0.0490047340628
DemAge : 0.0437416295918
DemCluster_11 : 0.0
StatusCat96NK_L : 0.0
StatusCat96NK_N : 0.0
StatusCat96NK_S : 0.0
DemCluster_0 : 0.0
DemCluster_1 : 0.0
DemCluster_10 : 0.0
DemCluster_13 : 0.0
DemCluster_12 : 0.0
StatusCat96NK_E : 0.0
DemCluster_14 : 0.0
DemCluster_15 : 0.0
DemCluster_16 : 0.0
DemCluster_17 : 0.0


True

![Simple decision tree structure](https://s3-ap-southeast-2.amazonaws.com/dataminingtuts/week3_dt_viz.png)

Now, the question is, how do we find the optimal combination of parameters for a model on a given dataset?

## 4. Finding optimal hyperparameters with GridSearchCV

A common method to find the optimal set of parameters for a model is to run a exhaustive search over all possible values of each parameter. To choose the best model among all candidates, k-fold cross validation is typically used.

In k-fold cross-validation, the training dataset is randomly partitioned into `k` equal size partitions. Of the k subsamples, a single subsample is retained as the validation data for testing the model, and the remaining k-1 subsamples are used as training data. The cross-validation process is then repeated k times (the folds), with each of the k subsamples used exactly once as the validation data. The k results from the folds can then be averaged (or otherwise combined) to produce a single estimation, which we will use to choose the best model. (courtesy of openml.org)

In `sklearn`, the exhaustive search + k-fold validation is implemented in `GridSearchCV`.

In [11]:
from sklearn.model_selection import GridSearchCV

To perform a GridSearchCV, we first have to determine the hyperparameters and possible values of parameters that we want to use. Each class of predictive models will have different kind of parameters (e.g. decision tree will be different to a regression). For this tutorial, we will search on 3 hyperparameters:
1. Criterion: The function to measure the quality of a split. There are two criterias we will use, “gini” for the Gini impurity and “entropy” for the information gain.
2. Max depth: The maximum depth of the tree. Let's start with range of 2-10.
3. Min samples leaf: The minimum number of samples required to be at a leaf node, allowing us to limit the minimum size of a leaf node. Let's start with range of 20-200 with step of 20.

In [12]:
# grid search CV
params = {'criterion': ['gini', 'entropy'],
          'max_depth': range(3, 10),
          'min_samples_leaf': range(20, 200, 20)}

cv = GridSearchCV(param_grid=params, estimator=DecisionTreeClassifier(), cv=10)
cv.fit(X_train, y_train)

# test the best model
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

# print parameters of the best model
print(cv.best_params_)

             precision    recall  f1-score   support

          0       0.54      0.73      0.62      2422
          1       0.59      0.39      0.47      2421

avg / total       0.57      0.56      0.55      4843

0.559157546975
{'max_depth': 4, 'criterion': 'gini', 'min_samples_leaf': 80}


We can see that the `accuracy` of our model increases slightly over the previous best. This is a promising sight, which means we might be able to further optimise on the found best parameters.

Let's do another grid search, now being more specific around the best params.

In [13]:
# grid search CV #2
params = {'criterion': ['gini', 'entropy'],
          'max_depth': range(2, 8),
          'min_samples_leaf': range(10, 30, 10)}

cv = GridSearchCV(param_grid=params, estimator=DecisionTreeClassifier(), cv=10)
cv.fit(X_train, y_train)

y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

print(cv.best_params_)

             precision    recall  f1-score   support

          0       0.56      0.62      0.59      2422
          1       0.58      0.51      0.54      2421

avg / total       0.57      0.57      0.57      4843

0.567210406773
{'max_depth': 2, 'criterion': 'gini', 'min_samples_leaf': 10}


Unfortunately, the `accuracy` does not improve further, which means for the 3 parameters we choose we have reached the best set. Try to experiment with other kinds of parameters and see whether you could do better than this. You can find the list of parameters available in a decision tree in `sklearn`'s `DecisionTreeClassifier` documentation website.

## Tips

1. Always perform feature importance and visualization to understand your decision tree. The best model can be found in `cv.best_estimator_`.
2. We will use feature importance a lot across this decision tree modelling process. Rather than writing the script multiple times (which is tedious and error-prone), we can just wrap them in functions in `dm_tools.py` and import it from there

In [14]:
# inside `dm_tools.py'
import numpy as np
import pydot
from io import StringIO
from sklearn.tree import export_graphviz

def analyse_feature_importance(dm_model, feature_names, n_to_display=20):
    # grab feature importances from the model
    importances = dm_model.feature_importances_
    
    # sort them out in descending order
    indices = np.argsort(importances)
    indices = np.flip(indices, axis=0)

    # limit to 20 features, you can leave this out to print out everything
    indices = indices[:n_to_display]

    for i in indices:
        print(feature_names[i], ':', importances[i])

def visualize_decision_tree(dm_model, feature_names, save_name):
    dotfile = StringIO()
    export_graphviz(dm_model, out_file=dotfile, feature_names=feature_names)
    graph = pydot.graph_from_dot_data(dotfile.getvalue())
    graph[0].write_png(save_name) # saved in the following file


In [15]:
# do the feature importance and visualization analysis on GridSearchCV's best model
from dm_tools import analyse_feature_importance, visualize_decision_tree

analyse_feature_importance(cv.best_estimator_, X.columns, 20)
visualize_decision_tree(cv.best_estimator_, X.columns, "dm_best_cv.png")

GiftAvgLast : 0.685126372799
GiftAvgCard36 : 0.173171593738
GiftTimeLast : 0.141702033462
DemGender_U : 0.0
DemCluster_11 : 0.0
StatusCat96NK_F : 0.0
StatusCat96NK_L : 0.0
StatusCat96NK_N : 0.0
StatusCat96NK_S : 0.0
DemCluster_0 : 0.0
DemCluster_1 : 0.0
DemCluster_10 : 0.0
DemCluster_13 : 0.0
DemCluster_12 : 0.0
StatusCat96NK_A : 0.0
DemCluster_14 : 0.0
DemCluster_15 : 0.0
DemCluster_16 : 0.0
DemCluster_17 : 0.0
DemCluster_18 : 0.0


![GridSearchCV decision tree](https://s3-ap-southeast-2.amazonaws.com/dataminingtuts/dm_best_cv.png)

## End notes and next week

This week, we learned how to perform data partitioning, build decision trees and test them, and finding the optimal parameter sets using GridSearchCV.

Next week, we will focus on performing predictive classification modelling using logistic regression.